In [1]:
import pandas as pd

In [2]:
#file to load
file_to_load="Resources/purchase_data.csv"

#read file and store into Pandas Data Frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#display the total number of unique players
total_players = len(purchase_data["SN"].unique())
total_players


576

In [4]:
#display the total number of unique in game items
unique_items = len(purchase_data["Item ID"].unique())
unique_items

179

In [5]:
#calculate average price per item
price_average = purchase_data["Price"].mean()
price_average

3.0509871794871795

In [6]:
#total revenue from all in game purchases
total_revenue=purchase_data["Price"].sum()
total_revenue


2379.77

In [7]:
#total number of purchases 
total_purchases=purchase_data["Purchase ID"].count()
total_purchases

780

In [8]:
#Summary Data Frame/cleaned up for organization 
summary_df_1=pd.DataFrame([{"Number of Unique Items": unique_items,
                           "Average Price": price_average,
                           "Number of Purchases":total_purchases,
                           "Total Revenue":total_revenue}])
summary_df_1

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [9]:
#Gender Demographics
gender_data=purchase_data.groupby("Gender")

#used the nunique function to get number of male female and other/non-disclosed players
gender_totals = gender_data["SN"].nunique()
gender_totals

#calculation for gender demo divided by total players
player_percent = gender_totals / total_players * 100

#New Data Frame
gender_totals=pd.DataFrame({"Total Count": gender_totals,"Percentage of Players": player_percent})
gender_totals.head()

,Total Count,Percentage of Players
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722


In [10]:
#Gender Demographics

gender_totals = gender_data["SN"].nunique()
gender_totals

Gender
Female                    81
Male                     484
Other / Non-Disclosed     11
Name: SN, dtype: int64

In [11]:
#Purchasing Analysis (Gender)
purch_count = gender_totals["Purchase ID"].count

average_purch=gender_totals["Price"].mean()

avg_purch_total=gender_totals["Price"].sum()

avg_per_person=avg_purch_total/gender_totals

gender_df=pd.DataFrame([{"Purchase Count": purch_count,
                       "Average Purchase Price": average_purch,
                       "Total Purchase Value": gender_totals,
                       "Avg Total Purchase Per Person": avg_per_person}])


KeyError: 'Purchase ID'

In [ ]:
##########

In [ ]:
#Age Demographics

#Find the min and max ages of the players to aid in creating bins
print(purchase_data["Age"].max())
print(purchase_data["Age"].min())


In [ ]:
bins= [0,10,15,20,25,30,35,40,100]

group_labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

#Slice the data and place it in bins 
pd.cut(purchase_data["Age"],bins, labels=group_labels).head()



In [ ]:
#finish the age demographics section

In [ ]:
#Purchasing Analysis Age 
bins= [0,10,15,20,25,30,35,40,100]

group_labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

#Slice the data and place it in bins 
pd.cut(purchase_data["Age"],bins, labels=group_labels).head()

In [ ]:
#Top Spenders

#view original data 
purchase_data=pd.DataFrame(purchase_data)
purchase_data.head()

#group data by the screen names
spend_by_screename=purchase_data.groupby("SN")

# count data by purchase id
purch_count_spender = spend_by_screename["Purchase ID"].count()

#get average purchase price
purchase_total = spend_by_screename["Price"].sum()

purchase_total.head()

